<a href="https://colab.research.google.com/github/steeze1213/Melon-Chart-Analysis/blob/main/Melon_Chart_Analysis/%EB%A9%9C%EB%A1%A0%EC%B0%A8%ED%8A%B8_%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요한 패키지 설치
!pip install transformers sentencepiece googletrans==4.0.0-rc1

# 라이브러리 불러오기
import pandas as pd
from transformers import pipeline
from googletrans import Translator

# 감정 영어-한글 변환 사전
emotion_translation = {
    "joy": "기쁨",
    "sadness": "슬픔",
    "anger": "분노",
    "fear": "두려움",
    "disgust": "혐오",
    "surprise": "놀라움",
    "anticipation": "기대"
}

# 번역기와 감정 분석 모델 초기화
translator = Translator()
emotion_analyzer = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=1)

# 감정 분석 함수 정의
def analyze_emotion(text):
    try:
        translated_text = translator.translate(text, src='ko', dest='en').text
        result = emotion_analyzer(translated_text)[0][0]
        return emotion_translation.get(result['label'], "중립")  # 감정 한글 변환
    except Exception as e:
        return "Error"

# 파일 경로 형식 및 연도 설정
base_path = '/content/drive/MyDrive/DS/최종 프로젝트/멜론차트/멜론{year}년 TOP30_20241104.csv'

# 2003년부터 2023년까지 파일 처리
for year in range(2003, 2024):
    # 파일 읽기
    file_path = base_path.format(year=year)
    data = pd.read_csv(file_path)

    # 감정 분석 수행 후 '대표 감정' 열 추가
    data['대표 감정'] = data['가사'].apply(analyze_emotion)

    # 분석 결과 저장
    output_path = f'/content/drive/MyDrive/DS/최종 프로젝트/멜론차트/감정분석_결과_{year}.csv'
    data.to_csv(output_path, index=False, encoding='utf-8-sig')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (535 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
# 필요 패키지 설치
!pip install transformers sentencepiece googletrans==4.0.0-rc1

# 라이브러리 불러오기
import pandas as pd
from transformers import pipeline
from googletrans import Translator

# 감정 영어-한글 변환 사전
emotion_translation = {
    "joy": "기쁨", "sadness": "슬픔", "anger": "분노",
    "fear": "두려움", "disgust": "혐오", "surprise": "놀라움",
    "anticipation": "기대", "neutral": "중립"
}

# 번역기 및 감정 분석 모델 초기화
translator = Translator()
emotion_analyzer = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)

# 감정 분석 함수 정의
def analyze_emotions(text):
    try:
        # 텍스트 번역
        translated_text = translator.translate(text, src='ko', dest='en').text
        # 감정 분석 수행
        results = emotion_analyzer(translated_text)[0]

        # 감정별 등장 횟수 초기화
        emotion_counts = {emotion: 0 for emotion in emotion_translation.values()}

        for item in results:
            emotion_name = emotion_translation.get(item['label'], "중립")
            if item['score'] > 0.5:  # 확률 임계값
                emotion_counts[emotion_name] += 1

        return emotion_counts

    except Exception as e:
        # 오류 발생 시 로그 출력 및 빈 데이터 반환
        print(f"Error processing text: {e}")
        return {emotion: 0 for emotion in emotion_translation.values()}

# 데이터 파일 불러오기
file_path = '/content/drive/MyDrive/DS/최종 프로젝트/멜론차트/멜론2003년 TOP30_20241104.csv'
data = pd.read_csv(file_path)

# '가사' 열에 대해 감정 분석 수행
emotions_data = data['가사'].apply(lambda x: pd.Series(analyze_emotions(x)))
data = pd.concat([data, emotions_data], axis=1)

# 결과 CSV 파일로 저장
output_path = '/content/drive/MyDrive/DS/최종 프로젝트/멜론차트/감정분석_결과_2003.csv'
data.to_csv(output_path, index=False, encoding='utf-8-sig')

감정 분석이 완료되었으며, 결과가 저장되었습니다.
